In [1]:
import pandas as pd;import math;import numpy as np; import geopandas as gpd; import matplotlib as plt

Load data

In [2]:
df1 = pd.read_csv("Data/siss_geo_indicatori_lineari_2016.csv", sep=",", encoding='utf-8')
df2 = pd.read_csv("Data/siss_geo_indicatori_lineari_2017.csv", sep=",", encoding='utf-8')
df3 = pd.read_csv("Data/siss_geo_indicatori_lineari_2018.csv", sep=",", encoding='utf-8')
df4 = pd.read_csv("Data/siss_geo_indicatori_lineari_2019.csv", sep=",", encoding='utf-8')
df5 = pd.read_csv("Data/siss_geo_indicatori_lineari_2020.csv", sep=",", encoding='utf-8')
df6 = pd.read_csv("Data/siss_geo_indicatori_lineari_2021.csv", sep=",", encoding='utf-8')
df7 = pd.read_csv("Data/siss_geo_indicatori_lineari_2022.csv", sep=",", encoding='utf-8')
df1['Year'] = 2016
df2['Year'] = 2017
df3['Year'] = 2018
df4['Year'] = 2019
df5['Year'] = 2020
df6['Year'] = 2021
df7['Year'] = 2022
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], sort=False)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/siss_geo_indicatori_lineari_2016.csv'

Create unique geolocation

In [ ]:
df['geometria'] = gpd.GeoSeries.from_wkt(df['geometria'])

In [ ]:
gdf1 = gpd.GeoDataFrame(df, geometry='geometria')
gdf1['length'] = gdf1.geometry.length
gdf1['centroid'] = gdf1.geometry.centroid
# fix uuids
gdf1['uid'] = gdf1['uuid'].str.upper()
gdf1['uid'] = gdf1['uid'].astype(str)                         # convert all to string
gdf1['uid'] = gdf1['uid'].str.strip()                         # remove leading/trailing whitespace
gdf1['uid'] = gdf1['uid'].str.replace(r'\s+', ' ', regex=True)  # normalise spaces
gdf1['uid'] = gdf1['uid'].str.replace(r'[^\x00-\x7F]+', '', regex=True)  # remove non-ASCII
# find longest geometry for each uuid
helper = gdf1.sort_values(by=['uid','length'], ascending=[True, False])
helper['maxgeometry'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['geometria'].shift(), helper['geometria'])#
helper['maxlength'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['length'].shift(), helper['length'])  
helper_0 = helper.groupby('uid', as_index=False).first()
gdf1['maxgeometry'] = gdf1['uid'].map(helper_0.set_index('uid')['maxgeometry']).fillna(0)
gdf1['maxlength'] = gdf1['uid'].map(helper_0.set_index('uid')['maxlength']).fillna(0)
# find accident for each uuid and find also max length for each uuid
accidents = gdf1[gdf1['nr_incidenti'] > 0]
helper = accidents.sort_values(by=['uid','length'], ascending=[True, False])
helper['maxgeometry_accidents'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['geometria'].shift(), helper['geometria'])#
helper['maxlength_accidents'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['length'].shift(), helper['length'])  
helper_0 = helper.groupby('uid', as_index=False).first()
gdf1['maxgeometry_accidents'] = gdf1['uid'].map(helper_0.set_index('uid')['maxgeometry_accidents']).fillna(0)
gdf1['maxlength_accidents'] = gdf1['uid'].map(helper_0.set_index('uid')['maxlength_accidents']).fillna(0)
# if accidents in multiple years, keep the one with the longest length
helper = accidents.groupby('uid', as_index=False, sort=False).agg({'geometria':'nunique','Year':'nunique','nr_incidenti':'sum'})
gdf1['accident_in_uuid'] = gdf1.uid.isin(helper.uid).astype(int)
acc_in_many_years_2 = helper[(helper['Year'] > 1) & (helper['geometria'] > 1)]
gdf1['acc_in_many_years'] = gdf1.uid.isin(acc_in_many_years_2.uid).astype(int)
gdf1['u_geometria'] = np.where(gdf1['accident_in_uuid'] == 1 , gdf1['maxgeometry_accidents'], gdf1['maxgeometry'])
gdf1['u_length'] = np.where(gdf1['accident_in_uuid'] == 1 , gdf1['maxlength_accidents'], gdf1['maxlength'])


Transformation Notes: 
1. Since there were road sections (uuid field) with multiple different geometria, we generate a unified geometria, together with its length, which can be found in the fields: u_geometria, u_length
2. Since the uuids change from capital to small letters, we create a unified reference for the uuids which can be found in the field: uid

Potentially useful fields for clustering analysis and/or explainability (e.g. predictive maintainance): 
1. area-related: 
    - fk_l3
    - fk_l2
    - cod_comune
    - group_strada
2. flow-related: 
    - flusso_medio_gior
    - flusso_medio_gior_s

Note: Those fields have missing values, so there are uids that do not have that info.

In [ ]:
tranformed_df = gdf1.drop(columns=['maxgeometry', 'maxlength','maxgeometry_accidents','maxlength_accidents','accident_in_uuid','acc_in_many_years'])
tranformed_df.head(n=20)

,uuid,fk_l3,es_amm_nom,fk_l2,tp_str_nom,cod_comune,desc_comune,flusso_medio_gior,length_el_str,nr_incidenti,...,class_line9_t,class_symbol9_t,class_line9_s,geometria,Year,length,centroid,uid,u_geometria,u_length
0,0A400430-D303-662B-E054-0003BA0F36E6,70560.0,SP 214,49755.0,VIA ROMA,4202.0,SALMOUR,705.0,NaN,0,...,0,0,0,"LINESTRING (404099.562 4936222.5, 404106.281 4...",2016,63.315767,POINT (404103.999 4936253.839),0A400430-D303-662B-E054-0003BA0F36E6,"LINESTRING (404099.56231678 4936222.49983281, ...",63.315767
1,093CB25B-83AD-2909-E054-0003BA0F36E6,70559.0,SP 213,213941.0,SP 213,4095.0,GARESSIO,500.0,NaN,0,...,0,0,0,"LINESTRING (423570.125 4894531, 423586.875 489...",2016,1142.104674,POINT (423891.973 4894505.532),093CB25B-83AD-2909-E054-0003BA0F36E6,"LINESTRING (423570.125150913 4894530.99976198,...",1142.104674
2,093CB25A-589D-2909-E054-0003BA0F36E6,0.0,NaN,21232.0,VIA INES MAGNONI,1301.0,VILLANOVA CANAVESE,1467.0,276.740,1,...,2,0,2,"LINESTRING (385798.625 5011371, 385854.187 501...",2016,276.740291,POINT (385896.019 5011274.028),093CB25A-589D-2909-E054-0003BA0F36E6,"LINESTRING (385798.625 5011371, 385854.187 501...",276.740427
3,0A400430-EC08-662B-E054-0003BA0F36E6,70560.0,SP 214,207208.0,STRADA PROVINCIALE,4232.0,TRINITA',3936.0,NaN,0,...,0,0,0,"LINESTRING (401452.156 4930481, 401471.219 493...",2016,42.066769,POINT (401461.687 4930499.75),0A400430-EC08-662B-E054-0003BA0F36E6,"LINESTRING (401452.156 4930481, 401471.219 493...",42.067184
4,093CB262-F73C-2909-E054-0003BA0F36E6,0.0,NaN,30690.0,VIA NOVARA,3024.0,BORGOMANERO,13626.0,175.454,3,...,1,0,1,"LINESTRING (459098.313 5059619.5, 459188.5 505...",2016,175.453979,POINT (459143.406 5059544.25),093CB262-F73C-2909-E054-0003BA0F36E6,"LINESTRING (459098.312576002 5059619.50013973,...",175.453979
5,0A400430-E410-662B-E054-0003BA0F36E6,70560.0,SP 214,207208.0,STRADA PROVINCIALE,4232.0,TRINITA',4088.0,NaN,0,...,0,0,0,"LINESTRING (401532.312 4930724.5, 401551.906 4...",2016,1208.819392,POINT (401705.019 4931303.605),0A400430-E410-662B-E054-0003BA0F36E6,"LINESTRING (401532.312394389 4930724.49995569,...",1208.819392
6,093CB266-7D99-2909-E054-0003BA0F36E6,0.0,NaN,17936.0,PIAZZALE SAN GABRIELE DI GORIZIA,1272.0,TORINO,13606.0,90.308,3,...,1,0,2,"LINESTRING (393983.406 4988213.5, 393926.969 4...",2016,90.307520,POINT (393955.188 4988178.25),093CB266-7D99-2909-E054-0003BA0F36E6,"LINESTRING (393983.406403359 4988213.49994523,...",90.307520
7,14625D41-B6F4-7198-E054-0003BA0F36E6,70562.0,SP 216,213988.0,SP 216,4155.0,ORMEA,500.0,NaN,0,...,0,0,0,"LINESTRING (411404.25 4885395, 411411.031 4885...",2016,440.461285,POINT (411428.354 4885191.738),14625D41-B6F4-7198-E054-0003BA0F36E6,"LINESTRING (411404.25 4885395, 411411.031 4885...",440.461759
8,093CB252-CB50-2909-E054-0003BA0F36E6,70562.0,SP 216,213988.0,SP 216,4155.0,ORMEA,500.0,NaN,0,...,0,0,0,"LINESTRING (412177.687 4886039.5, 412179.094 4...",2016,305.953886,POINT (412274.933 4885985.048),093CB252-CB50-2909-E054-0003BA0F36E6,"LINESTRING (412177.687 4886039.5, 412179.094 4...",305.954192
9,093CB25C-ABF2-2909-E054-0003BA0F36E6,0.0,NaN,18009.0,STRADA al TRAFORO DI PINO,1272.0,TORINO,13821.0,115.580,1,...,1,0,1,"LINESTRING (401967.25 4991421, 401997.969 4991...",2016,115.579767,POINT (402015.851 4991389.777),093CB25C-ABF2-2909-E054-0003BA0F36E6,"LINESTRING (401967.250250794 4991421.00023622,...",115.579767


In [ ]:
tranformed_df.to_csv("mydata.csv", sep=",", encoding='utf-8', index=False)
# tranformed_df[0:math.floor(tranformed_df.shape[0]/2)].to_csv("accidents_transformed_1.csv", sep=",", encoding='utf-8', index=False)
# tranformed_df[math.floor(tranformed_df.shape[0]/2)+1:].to_csv("accidents_transformed_2.csv", sep=",", encoding='utf-8', index=False)